#  📥 Scraping de Booking

## 🧰 Librerías e importaciones

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [ ]:
from selenium.common.exceptions import NoSuchElementException
import csv

In [ ]:
import time

In [ ]:
import re

In [ ]:
from selenium.common.exceptions import ElementClickInterceptedException

## 🔧 Funciones auxiliares

In [ ]:
def handle_no_such_element_exception(data_extraction_task):
    try:
        return data_extraction_task()
    except NoSuchElementException as e:
        return None

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

def auto_scroll(driver, pause_time=2, max_scrolls=10):
    last_height = driver.execute_script("return document.body.scrollHeight")

    for i in range(max_scrolls):
        time.sleep(0.3)
        print(f"Haciendo scroll {i+1}/{max_scrolls}...")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("No se cargó más contenido.")
            break
        last_height = new_height

def paginar_todas(driver, delay=3, max_pages=20):
    from selenium.common.exceptions import ElementClickInterceptedException
    page = 1
    while page <= max_pages:
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//button[.//span[text()="Ver más"]]'))
            )
            print(f"Haciendo clic en página {page}")
            next_button.click()
            time.sleep(delay)
            page += 1
        except:
            print("No hay más páginas o no se pudo hacer clic.")
            break

In [ ]:
def scroll_and_click_all(driver, max_cycles=30, delay=3, max_scrolls=10, pause_time=2):
    last_height = driver.execute_script("return document.body.scrollHeight")
    page = 1

    for cycle in range(max_cycles):
        print(f"→ Ciclo {cycle + 1}/{max_cycles} de scroll + clic")

        # Scroll al fondo varias veces
        for i in range(max_scrolls):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print("📉 No se cargó más contenido al hacer scroll.")
                break
            last_height = new_height

        try:
            # Asegurarse de que el botón "Ver más" esté presente y visible
            next_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//button[.//span[text()="Ver más"]]'))
            )

            # Moverse al botón para que esté en vista
            try:
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                time.sleep(1)
                print("Intentando hacer clic en 'Ver más'...")

                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//button[.//span[text()="Ver más"]]')))
                next_button.click()
                print(f"✅ Clic exitoso en página {page}")
                page += 1
                time.sleep(delay)

            except ElementClickInterceptedException:
                print("⚠️ ElementClickInterceptedException: Intentando con JavaScript...")
                driver.execute_script("arguments[0].click();", next_button)
                time.sleep(delay)
                page += 1

        except TimeoutException:
            print("⛔ Timeout: No se encontró el botón 'Ver más'. Probablemente ya no hay más.")
            break
        except NoSuchElementException:
            print("⛔ NoSuchElementException: No hay más botón 'Ver más'")
            break

## 🚗 Driver y utilidades de navegación

In [ ]:
def close_all_known_popups(driver):
    popups_closed = 0
    try:
        # Popup de inicio de sesión
        close_button = driver.find_element(By.CSS_SELECTOR, '[role="dialog"] button[aria-label="Dismiss sign-in info."]')
        close_button.click()
        print("Popup de inicio de sesión cerrado.")
        popups_closed += 1
    except:
        pass

    try:
        # Popup de "Instala nuestra app"
        app_popup = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Cerrar ventana emergente"]')
        app_popup.click()
        print("Popup de app cerrado.")
        popups_closed += 1
    except:
        pass

    try:
        # Otro tipo genérico de modal (ajustable según el sitio)
        generic_popup = driver.find_element(By.CSS_SELECTOR, '.modal-mask button')
        generic_popup.click()
        print("Popup genérico cerrado.")
        popups_closed += 1
    except:
        pass

    if popups_closed == 0:
        print("No se detectaron popups.")


## 🕷️ Scraper de una ciudad y exportación CSV

In [ ]:
driver = webdriver.Chrome(service=Service())

driver.get("https://www.booking.com/attractions/searchresults/es/barcelona.es.html?adplat=www-index-web_shell_header-attraction-missing_creative-1zwEmyAV9Sm0av9pzjIUKp&label=gen173nr-1BCAEoggI46AdIM1gEaEaIAQGYAQq4ARfIAQzYAQHoAQGIAgGoAgO4AuSv-sMGwAIB0gIkMzQzOWUzMjItNGVmMC00NmVjLTg0NTAtMWVlMGI1OWMzMzU02AIF4AIB&distribution_id=1zwEmyAV9Sm0av9pzjIUKp&aid=304142&client_name=b-web-shell-bff&source=search_box&start_date=2025-07-21&end_date=2026-01-16")
close_all_known_popups(driver)
# Aceptar cookies si aparece el banner
try:
    accept_cookies_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
    )
    accept_cookies_btn.click()
    print("Cookies aceptadas.")
except:
    print("No apareció el banner de cookies.")

# handle the sign-in alert
try:
    # wait up to 20 seconds for the sign-in alert to appear
    close_button = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '[role="dialog"] button[aria-label="Dismiss sign-in info."]'))
    )
    # click the close button
    close_button.click()
except:
    print("Sign-in modal did not appear, continuing...")

scroll_and_click_all(driver, max_cycles=70, delay = 5, max_scrolls= 30)
close_all_known_popups(driver)
# where to store the scraped data
items = []

# select all property items on the page
property_items = driver.find_elements(By.CSS_SELECTOR, '[data-testid="card"]')

# iterate over the property items and
# extract data from them
i = 0

for idx, property_item in enumerate(property_items):
    close_all_known_popups(driver)
    # Scroll hacia la tarjeta individual para asegurarse de que cargue
    try:
        ActionChains(driver).move_to_element(property_item).perform()
        time.sleep(1.5)
    except:
        print("No se pudo hacer scroll a la tarjeta.")

    # Esperar explícitamente a que haya imagen u otro contenido

    url = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, 'a').get_attribute("href"))


    title = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, '[data-testid="card-title"]').text)


    spans = property_item.find_elements(By.CSS_SELECTOR, 'span[aria-hidden="true"]')
    review_score = None
    review_count = None

    for span in spans:
        text = span.text.strip()
        if not text:
            continue

        # Si es un número como 4,6 o 4.7 (review score)
        if text.replace(",", "").replace(".", "").isdigit() and "," in text:
            review_score = float(text.replace(",", "."))

        # Si contiene la palabra "comentarios"
        elif "comentarios" in text.lower():
            try:
                review_count = int(text.split()[0].replace(".", "").replace(",", ""))
            except:
                continue
    price_text = handle_no_such_element_exception(
    lambda: property_item.find_element(By.CSS_SELECTOR, '[data-testid="price"]').text
    )
    # if not price_text:
    #     divs = property_item.find_elements(By.TAG_NAME, 'div')
    #     for div in divs:
    #         text = div.text.strip()
    #         if "€" in text:
    #             price_text = text
    #             break
    # if price_text:
    #     price = float(price_text.replace("€", "").replace(",", ".").strip())
    # else:
    #     price = None

    # print(price)
    i = i+1
    print(f"Se hizo el cuadro numero {i}")
    # populate a new item with the scraped data
    item = {
      "url": url,
      "title": title,
      "review_score": review_score,
      "review_count": review_count,
      "price": price_text
    }
    # add the new item to the list of scraped items
    items.append(item)

# specify the name of the output CSV file
output_file = "atracciones_barcelona2.csv"

# export the items list to a CSV file
with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    #create a CSV writer object
    writer = csv.DictWriter(file, fieldnames=["url",  "title", "review_score", "review_count", "price"])

    # write the header row
    writer.writeheader()

    # write each item as a row in the CSV
    writer.writerows(items)

# close the web driver and release its resources
driver.quit()

No se detectaron popups.
Cookies aceptadas.
Sign-in modal did not appear, continuing...
→ Ciclo 1/70 de scroll + clic
📉 No se cargó más contenido al hacer scroll.
Intentando hacer clic en 'Ver más'...
✅ Clic exitoso en página 1
→ Ciclo 2/70 de scroll + clic
📉 No se cargó más contenido al hacer scroll.
Intentando hacer clic en 'Ver más'...
✅ Clic exitoso en página 2
→ Ciclo 3/70 de scroll + clic
📉 No se cargó más contenido al hacer scroll.
Intentando hacer clic en 'Ver más'...
✅ Clic exitoso en página 3
→ Ciclo 4/70 de scroll + clic
📉 No se cargó más contenido al hacer scroll.
Intentando hacer clic en 'Ver más'...
✅ Clic exitoso en página 4
→ Ciclo 5/70 de scroll + clic
📉 No se cargó más contenido al hacer scroll.
Intentando hacer clic en 'Ver más'...
✅ Clic exitoso en página 5
→ Ciclo 6/70 de scroll + clic
📉 No se cargó más contenido al hacer scroll.
Intentando hacer clic en 'Ver más'...
✅ Clic exitoso en página 6
→ Ciclo 7/70 de scroll + clic
📉 No se cargó más contenido al hacer scrol